In [1]:
from cgmbrush import *
import cProfile
import io
import pstats

In [2]:
# Perf testing for reading Bolshoi density fields

provider = BolshoiProvider()

pr = cProfile.Profile()
pr.enable()
den_field = provider.get_density_fields([0], 256)
pr.disable()
s = io.StringIO()
ps = pstats.Stats(pr, stream=s).sort_stats('tottime')
ps.print_stats()

with open('bolshoi_density_perf.txt', 'w+') as f:
    f.write(s.getvalue())



TypeError: __init__() got an unexpected keyword argument 'steam'